In [20]:

import holoviews as hv
import panel as pn
import numpy as np
import pandas as pd
import geopandas as gpd
import re
from panel.template import FastGridTemplate
from panel.template import DarkTheme
import hvplot.pandas
import param

MAIN = "#D6ADD0"
ACCENT = "#724175"


In [21]:
df = pd.read_csv("./data/trips_by_year.csv", sep=",")
df2 = pd.read_csv("./data/trips_by_month.csv", sep=",")
df3 = pd.read_csv("./data/trips_by_hour.csv", sep=",")
df3["mean"] = df3["trips"].mean()

In [42]:
#### working with kdims 
def _create_barandlinewidget(data,title, x, xlabel):
   
    def plot(column="duration(year)"):
        def hooks(plot, element):
            p = plot.state
            p.toolbar.autohide = True
            p.toolbar.logo = None
            if "trips" in column:
                p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
                p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
            else:
                p.yaxis.ticker = np.arange(0, 1400, 200)
                p.yaxis.major_label.overrides={0:"0", 200:"200", 400:"400", 600:"600", 800:"800", 1000:"1000",1200:"1200", 1400:"1400"}
    
        return data.hvplot(kind="bar",
                           x=x,
                           title=title,
                           y=column,
                           color=MAIN,
                           xlabel=xlabel,
                           line_color="white",
                           responsive=True,
                           min_width=350,
                           min_height=250,
                           ylabel=re.sub(r"\([a-z,A-Z]*\)","", str(column)).capitalize(),
                           label=re.sub(r"\([a-z,A-Z]*\)","", str(column)).capitalize()).opts(legend_position='top', hooks=[hooks]) \
                               * data.hvplot(kind="line",
                                           x=x,
                                           y=f"mean{column}",
                                           responsive=True, 
                                           line_color="white",
                                           label="Mean")

    dmap = hv.DynamicMap(plot, kdims=["column"],sizing_mode="stretch_both").redim.values(column=[f"duration({x})", f"trips({x})"])
    hv_panel = pn.panel(dmap)
    widget = hv_panel[1]
    return pn.Column(
        pn.Row(*widget),
        hv_panel[0],
        
        sizing_mode="stretch_both",
    )

In [43]:
def _create_barandline(data,title, x, y, y2, xlabel, ylabel, label, label2):
    def hooks(plot, element):
        p = plot.state
        p.toolbar.autohide = True
        p.toolbar.logo = None
        p.yaxis.ticker=[0,2000000,4000000,6000000,8000000]
        p.yaxis.major_label_overrides={0:"0",2000000:"2M",4000000:"4M",6000000:"6M",8000000:"8M"} 
        p.toolbar_location = "above"
 
    def plot():
        return data.hvplot.bar(title=title,
                               x=x,
                               xlabel=xlabel,
                               ylabel=ylabel,
                               y=y,
                               color=MAIN,
                               responsive=True,
                               line_color="white",
                               min_width=350,
                               min_height=250,
                               label=label).opts(legend_position="right",
                               hooks=[hooks]) \
        * df3.hvplot.line(x=x,
                          y=y2,
                          color=ACCENT,
                          responsive=True,
                          min_width=300,
                          min_height=150,
                          line_color="white",
                          label=label2)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(plot), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )

In [44]:
def _create_map():
    df = pd.read_csv("./data/coordinatesGeo.csv")
    dfgeo = gpd.GeoDataFrame(
    df.loc[:, [c for c in df.columns if c != "geometry"]],
    geometry=gpd.GeoSeries.from_wkt(df["geometry"]),
    crs="epsg:3005")
    def map():
        return dfgeo.hvplot(geo=True, hover_cols='Name',tiles=True, xaxis=None, yaxis=None, color=ACCENT, responsive=True)
    return pn.Column(
        pn.pane.HoloViews(hv.DynamicMap(map), sizing_mode="stretch_both"),
                sizing_mode="stretch_both",
    )

In [38]:
# maybe use geoviews in the future instead of hvplot for the map
def hooks(plot, element):
            p = plot.state
            p.toolbar.autohide= True
            p.toolbar.logo = None

def create_mapymcmapface(url):
    class Map(param.Parameterized):
        data = pd.read_csv(url)
        value = param.Integer(default=97, bounds=(min(data["value"]), 300000))
        @param.depends('value')
        def create_map(self):
            df = self.data.loc[self.data["value"] >= self.value]
            return df.hvplot.points('lon', 'lat', title="Most popular stations to start journeys",
                                    geo=True, tiles=True, hover_cols=["station","value"],xaxis=None, yaxis=None, responsive=True).opts(hooks=[hooks], framewise=True)
        @param.depends("value")
        def view(self):
            map = hv.DynamicMap(self.create_map)
            return map
        
    map = Map()
    return pn.Column(
        pn.Param(map.param, name="Foo"),
        map.view, sizing_mode="stretch_both")
    


In [45]:
ACCENT = "#1c1c1c"
template = FastGridTemplate(
    title="London TFL Bike Journeys Dashboard",
    row_height=55,
    prevent_collision=True,
    save_layout=True,
    accent_base_color=ACCENT,
    header_background=ACCENT,
    header_accent_base_color="Light Coral",
    theme=DarkTheme,
    theme_toggle=False
)

In [46]:
template.main[:7, 0:6] = _create_map()
template.main[:5, 6:11] = _create_barandlinewidget(data=df2, title="Trips by month", x="month", xlabel="Month")
template.main[5:6, 6:11] = pn.pane.Markdown("      # Total Trips: 76450245",style={"margin-top":"-30px", "padding":"0,0,0,0"}, width=400, height=0, )
template.main[7:12, 0:11] = _create_barandline(data=df3, title="Trips by time of day", x="time", y="trips", y2="mean", xlabel="Time of day", ylabel="Trips (in millions)", label="Trips", label2="Mean")
template.main[12:18, 0:5] = _create_barandlinewidget(data=df, title="Trips by year", x="year", xlabel="Year")
template.main[18:19,0:5] = pn.Row(pn.pane.Markdown("#    🚴🚴🚴   Over 800 Docks   🚴🚴🚴",style={"margin-top": "-25px"}, width=400, height=10), scroll=False)
template.main[12:20,5:11] = create_mapymcmapface("./data/stations_start.csv")
template.main[20:20, 0:12] = pn.Spacer()

In [47]:
template.servable()
template.show()
template.save("./dashboardFastGrid.html"    )

Launching server at http://localhost:32849



 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font

 unexpected attribute 'major_label' to LinearAxis, similar attributes are major_label_policy, major_label_standoff or major_label_text_font
